## <b><font color='darkblue'>Preface</font></b>
([course source](https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/2/build-an-agent-from-scratch))

![react-idea](images/ch2_1.PNG)
([diagram source](https://react-lm.github.io/))

In [1]:
# based on https://til.simonwillison.net/llms/python-react-pattern

In [2]:
!pip freeze | grep -P '(openai|langchain)'

langchain==0.2.6
langchain-anthropic==0.1.15
langchain-community==0.2.6
langchain-core==0.2.10
langchain-experimental==0.0.62
langchain-google-genai==1.0.6
langchain-groq==0.1.3
langchain-openai==0.1.9
langchain-text-splitters==0.2.0
langchainhub==0.1.14
openai==1.28.1


In [52]:
import os
import openai
import re
import httpx
import os
from dotenv import load_dotenv, find_dotenv

from openai import OpenAI
from langchain_openai import ChatOpenAI

a = load_dotenv(find_dotenv(os.path.expanduser('~/.env'))) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [5]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [19]:
#llm = ChatOpenAI(temperature=0, model=llm_model)

In [20]:
client = OpenAI()

In [21]:
chat_completion = client.chat.completions.create(
    model=llm_model,
    messages=[{"role": "user", "content": "Hello world"}]
)
chat_completion.choices[0].message.content

'Hello! How are you today?'

## <b><font color='darkblue'>Agent v0</font></b>
<b><font size='3ptx'>Our first agent can remember the execution or chat history. However, it still doesn't not be able to execute the tools/functions by itself yet.</font></b>

Let's try the execution follow of a client:

In [23]:
prompt = 'You are a chatbot. When you finish answer the question, you always append the message "Thanks for asking!"'
messages = [{"role": "system", "content": prompt}]

In [24]:
# 1. Prepare first message:
message = 'Nice to meet you. I am John.'
messages.append({"role": "user", "content": message})

In [25]:
# 2. Post the message to LLM
resp = client.chat.completions.create(
    model=llm_model, 
    temperature=0,
    messages=messages)

In [26]:
resp

ChatCompletion(id='chatcmpl-9fltsnfKpCUQkDb4sV15UijTAxPWj', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Nice to meet you too, John! How can I assist you today? Thanks for asking!', role='assistant', function_call=None, tool_calls=None))], created=1719742604, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=19, prompt_tokens=43, total_tokens=62))

In [29]:
# 3. Retrieve the responded message from LLM
resp.choices[0].message.content

'Nice to meet you too, John! How can I assist you today? Thanks for asking!'

In [30]:
# 4. Add the obtained message back to `messages` list
messages.append({"role": "assistant", "content": resp.choices[0].message.content})

In [31]:
# 5. Post the next message and because of the history of messages, LLM could know who you are
message = 'Do you know who I am?'
messages.append({"role": "user", "content": message})

In [32]:
# 6. Get the response and print the obtained message:
resp = client.chat.completions.create(
    model=llm_model, 
    temperature=0,
    messages=messages)
print(resp.choices[0].message.content)

Yes, you mentioned your name is John. How can I help you, John? Thanks for asking!


Now, let's create our first agent from scratch:

In [33]:
class Agent:
    def __init__(self, system: str="", llm_model: str="gpt-4o"):
        self.system = system
        self.llm_model = llm_model
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model=self.llm_model, 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content

Then below is the prompt used by our agent:

In [34]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

Below are supported/known functions for LLM to leverage:

In [36]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

Alright, let's test our first agent to see how it works:

In [37]:
abot = Agent(prompt)

In [38]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the average weight of a toy poodle using the average_dog_weight action.
Action: average_dog_weight: Toy Poodle
PAUSE


As requested from the `prompt`, LLM knows in order to get the weight of toy poodle, it needs to execute the function `average_dog_weight` and feed in the argument `Toy Poodle`! For now, let's execute the function on behavior of the LLM:

In [39]:
result = average_dog_weight("Toy Poodle")

In [40]:
result

'a toy poodles average weight is 7 lbs'

Next, let's compose the response as `"Observataion"` and feed back into our agent:

In [41]:
next_prompt = "Observation: {}".format(result)
next_prompt

'Observation: a toy poodles average weight is 7 lbs'

In [42]:
abot(next_prompt)

'Answer: A toy poodle weighs an average of 7 lbs.'

As expected, we got the message with prefix `Answer:` which means the message is the final answer to our question.

Let's check all the messages accumulated so far:

In [43]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

Let's do one more time but with a more complex question.

In [44]:
abot = Agent(prompt)

Ask the question which requires more than one actions in order to get the final answer:

In [45]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: I need to find the average weight of both a Border Collie and a Scottish Terrier, then sum these weights to get the combined weight of the two dogs.\nAction: average_dog_weight: Border Collie\nPAUSE'

Let's call the help function `average_dog_weight` on behavior of LLM to get the weight of `Border Collie`:

In [46]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


Next, let's feed the obtained observation back to LLM:

In [47]:
abot(next_prompt)

'Thought: Now I need to find the average weight of a Scottish Terrier.\nAction: average_dog_weight: Scottish Terrier\nPAUSE'

Again, let's call the function `average_dog_weight` on behavior of LLM to get the weight of `Scottish Terrier`:

In [48]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


and feed in the obtained observation back to LLM too:

In [49]:
abot(next_prompt)

'Thought: I now have the average weights of both dogs. I need to sum these weights to get the combined weight.\nAction: calculate: 37 + 20\nPAUSE'

This time, the execution of function `calculate` is required to sum up the weights. Let's do it:

In [50]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


As usual, let's feed in the observation back to LLM:

In [51]:
abot(next_prompt)

'Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.'

No surprise, we got the correct answer `57 lbs` back. Cool! isn't it?

## <b><font color='darkblue'>Agent v1</font></b>
<b><font size='3ptx'>We are going to enable the agent to be capable of executing the tools/functions now.</font></b>

### <b><font color='darkgreen'>RE to extract the function name and corresponding arguments for execution</font></b>

In [53]:
# Python regular expression to selection action
action_re = re.compile('^Action: (\w+): (.*)$')

### <b><font color='darkgreen'>Query function</font></b>
Let's put all the things we have done together and wrap them up as a function `query`:

In [55]:
def query(question: str, max_turns: int=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

Let's test it with the question we just asked before:

In [57]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I need to find the average weight of both a Border Collie and a Scottish Terrier, then add them together to get the combined weight.
Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Now I need to find the average weight of a Scottish Terrier.
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: I now have the average weights of both dogs. I will add them together to find their combined weight.
Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.


## <b><font color='darkblue'>Supplement</font></b>
* [Deeplearn.ai - AI Agents in LangGraph - Ch3: LangGraph Components](https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/3/langgraph-components)
* [Paper - ReAct: Synergizing Reasoning and Acting in Language Models](https://react-lm.github.io/)
> Language models are getting better at reasoning (e.g. chain-of-thought prompting) and acting (e.g. WebGPT, SayCan, ACT-1), but these two directions have remained separate. ReAct asks, what if these two fundamental capabilities are combined?